# 日立ISPデータの前処理（両眼データ版）

日立から提供されたISPデータセット（`datasets/hitachi/ISP.csv`）から両眼データのみを抽出して整理する。

## データ構造
- 検査眼が「両眼」のデータのみを処理
- 列53から右眼28測定点（各測定点は5列：X座標、Y座標、見えた/見えない、閾値、誤差）
- その後に左眼28測定点（同じ構造）
- TRUE/FALSEを1/0に変換
- 出力は右眼と左眼を別行として、基本情報 + point1〜point28形式

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# pandasの列の表示数を増やす
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# ルートに移動
%cd /Users/keimy/git/DeepISP
%pwd

# ISPデータを読み込み
print("=== ISPデータの読み込み ===")
df_isp = pd.read_csv('datasets/hitachi/ISP.csv', encoding='cp932')
print(f"読み込み完了: {len(df_isp)}行")

# coord_isp.csvを読み込み（標準ISP座標）
coord_isp = pd.read_csv('datasets/solid/coord_isp.csv')
print(f"標準座標読み込み完了: {len(coord_isp)}点")

/Users/keimy/git/DeepISP
=== ISPデータの読み込み ===
読み込み完了: 19367行
標準座標読み込み完了: 28点


In [18]:
# 検査眼列を探してフィルタリング
print("=== 検査眼の種類を確認 ===")

# 検査眼列を探す
eye_col = None
for i, col in enumerate(df_isp.columns[:20]):  # 最初の20列を確認
    if '検査眼' in col and 'X座標' not in col and 'Y座標' not in col:
        eye_col = col
        print(f"検査眼列: 列{i} ({col})")
        break

if eye_col:
    # 検査眼の種類と数を集計
    eye_counts = df_isp[eye_col].value_counts()
    print(f"\n検査眼の分布:")
    for eye_type, count in eye_counts.items():
        print(f"  {eye_type}: {count}件")
    
    # 両眼データのみをフィルタリング
    df_isp_binocular = df_isp[df_isp[eye_col].str.contains('両眼', na=False)]
    print(f"\n両眼データのみを抽出: {len(df_isp_binocular)}行")
    
    # 以降の処理はdf_isp_binocularを使用
    df_isp = df_isp_binocular
    print(f"処理対象データ: {len(df_isp)}行")
else:
    print("検査眼列が見つかりませんでした")

=== 検査眼の種類を確認 ===
検査眼列: 列6 (検査眼)

検査眼の分布:
  両眼: 18379件
  右眼: 498件
  左眼: 490件

両眼データのみを抽出: 18379行
処理対象データ: 18379行


In [19]:
# 正しい測定点の開始位置を特定（列53から）
print("=== 測定点の正しい構造 ===")

# 測定点1は列53「X座標：-27」から始まる（盲点は無視）
start_col = 53
print(f"測定点1（point1）の開始位置: 列{start_col}")
print(f"列名: {df_isp.columns[start_col]}")

# 測定点を収集（列53から5列ごと、28点ずつ）
right_eye_sets = []
left_eye_sets = []

# 右眼の28測定点（列53から）
col_idx = start_col
for i in range(28):
    right_eye_sets.append({
        'point_num': i + 1,
        'x_idx': col_idx,
        'y_idx': col_idx + 1,
        'vis_idx': col_idx + 2,  # 見えた/見えない列
        'threshold_idx': col_idx + 3,
        'error_idx': col_idx + 4
    })
    col_idx += 5  # 次の測定点へ（5列ごと）

# 左眼の28測定点（右眼の後）
for i in range(28):
    left_eye_sets.append({
        'point_num': i + 1,
        'x_idx': col_idx,
        'y_idx': col_idx + 1,
        'vis_idx': col_idx + 2,  # 見えた/見えない列
        'threshold_idx': col_idx + 3,
        'error_idx': col_idx + 4
    })
    col_idx += 5  # 次の測定点へ（5列ごと）

print(f"\n右眼測定点数: {len(right_eye_sets)}")
print(f"左眼測定点数: {len(left_eye_sets)}")

# 最初の5測定点を確認
print("\n最初の5測定点（右眼）:")
for i in range(5):
    mset = right_eye_sets[i]
    print(f"\npoint{mset['point_num']}:")
    print(f"  列{mset['x_idx']}: {df_isp.columns[mset['x_idx']]}")
    print(f"  列{mset['y_idx']}: {df_isp.columns[mset['y_idx']]}")
    print(f"  列{mset['vis_idx']}: {df_isp.columns[mset['vis_idx']]}")

# coord_ispとの対応を確認
print("\n=== coord_ispとの対応確認 ===")
coord_isp_28 = coord_isp.iloc[:28]
print("coord_ispの最初の5点:")
for i in range(5):
    print(f"  point{i+1}: ({coord_isp_28.iloc[i]['x']}, {coord_isp_28.iloc[i]['y']})")

=== 測定点の正しい構造 ===
測定点1（point1）の開始位置: 列53
列名: X座標：-27

右眼測定点数: 28
左眼測定点数: 28

最初の5測定点（右眼）:

point1:
  列53: X座標：-27
  列54: Y座標：+3
  列55: 見えた/見えない

point2:
  列58: X座標：-27.1
  列59: Y座標：-3
  列60: 見えた/見えない.1

point3:
  列63: X座標：-21
  列64: Y座標：+3.1
  列65: 見えた/見えない.2

point4:
  列68: X座標：-21.1
  列69: Y座標：-3.1
  列70: 見えた/見えない.3

point5:
  列73: X座標：-15
  列74: Y座標：+9
  列75: 見えた/見えない.4

=== coord_ispとの対応確認 ===
coord_ispの最初の5点:
  point1: (-27, 3)
  point2: (-27, -3)
  point3: (-21, 3)
  point4: (-21, -3)
  point5: (-15, 9)


In [20]:
# 左眼の座標を確認（X座標の反転を考慮）
print("\n=== 左眼の座標確認 ===")
print("左眼の最初の5測定点:")
for i in range(5):
    mset = left_eye_sets[i]
    if mset['x_idx'] < len(df_isp.columns):
        x_col = df_isp.columns[mset['x_idx']]
        y_col = df_isp.columns[mset['y_idx']]
        print(f"\n左眼point{i+1}:")
        print(f"  列{mset['x_idx']}: {x_col}")
        print(f"  列{mset['y_idx']}: {y_col}")
        
        # 実際の座標値を確認
        if len(df_isp) > 0:
            x_val = df_isp.iloc[0, mset['x_idx']]
            y_val = df_isp.iloc[0, mset['y_idx']]
            print(f"  実データ: X={x_val}, Y={y_val}")
            
            # X座標反転後の値
            print(f"  X反転後: X={-x_val}, Y={y_val}")
            
            # 対応する右眼座標を探す
            for j, coord in enumerate(coord_isp_28.itertuples()):
                if coord.x == -x_val and coord.y == y_val:
                    print(f"  → 右眼フォーマットのpoint{j+1}に対応")


=== 左眼の座標確認 ===
左眼の最初の5測定点:

左眼point1:
  列193: X座標：-9.2
  列194: Y座標：+9.3
  実データ: X=-9.0, Y=9.0
  X反転後: X=9.0, Y=9.0
  → 右眼フォーマットのpoint27に対応

左眼point2:
  列198: X座標：-9.3
  列199: Y座標：-9.3
  実データ: X=-9.0, Y=-9.0
  X反転後: X=9.0, Y=-9.0
  → 右眼フォーマットのpoint28に対応

左眼point3:
  列203: X座標：-7.2
  列204: Y座標：+5.2
  実データ: X=-7.0, Y=5.0
  X反転後: X=7.0, Y=5.0
  → 右眼フォーマットのpoint25に対応

左眼point4:
  列208: X座標：-7.3
  列209: Y座標：-5.2
  実データ: X=-7.0, Y=-5.0
  X反転後: X=7.0, Y=-5.0
  → 右眼フォーマットのpoint26に対応

左眼point5:
  列213: X座標：-3.4
  列214: Y座標：+15.2
  実データ: X=-3.0, Y=15.0
  X反転後: X=3.0, Y=15.0
  → 右眼フォーマットのpoint21に対応


In [21]:
def extract_eye_data(row, measurement_sets):
    """28測定点のデータを抽出（TRUE/FALSE → 1/0）"""
    data = []
    
    for i, mset in enumerate(measurement_sets):
        # 見えた/見えない列から値を取得
        vis_val = row.iloc[mset['vis_idx']]
        
        # TRUE/FALSEを1/0に変換
        if pd.notna(vis_val):
            if str(vis_val).upper() == 'TRUE':
                data.append(1)
            elif str(vis_val).upper() == 'FALSE':
                data.append(0)
            else:
                data.append(np.nan)
        else:
            data.append(np.nan)
    
    return data

# テスト: 最初の数行でデータを抽出
print("=== データ抽出テスト ===")
for i in range(min(3, len(df_isp))):
    # 右眼データ
    right_data = extract_eye_data(df_isp.iloc[i], right_eye_sets)
    valid_right = [d for d in right_data if pd.notna(d)]
    
    print(f"\n行{i+1}:")
    print(f"  右眼: 有効データ{len(valid_right)}/28, "
          f"見えた={sum(d==1 for d in valid_right)}, "
          f"見えない={sum(d==0 for d in valid_right)}")
    print(f"  最初の10点: {right_data[:10]}")
    
    # 左眼データ（ある場合）
    if left_eye_sets:
        left_data = extract_eye_data(df_isp.iloc[i], left_eye_sets)
        valid_left = [d for d in left_data if pd.notna(d)]
        print(f"  左眼: 有効データ{len(valid_left)}/28, "
              f"見えた={sum(d==1 for d in valid_left)}, "
              f"見えない={sum(d==0 for d in valid_left)}")

=== データ抽出テスト ===

行1:
  右眼: 有効データ28/28, 見えた=28, 見えない=0
  最初の10点: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  左眼: 有効データ28/28, 見えた=28, 見えない=0

行2:
  右眼: 有効データ28/28, 見えた=28, 見えない=0
  最初の10点: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  左眼: 有効データ28/28, 見えた=28, 見えない=0

行3:
  右眼: 有効データ28/28, 見えた=28, 見えない=0
  最初の10点: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  左眼: 有効データ28/28, 見えた=28, 見えない=0


## 3. データ処理と保存

In [22]:
# 基本情報のカラムマッピング
basic_cols = {
    df_isp.columns[0]: 'daily_id',
    df_isp.columns[1]: 'date',
    df_isp.columns[2]: 'time',
    df_isp.columns[3]: 'exam_id',
    df_isp.columns[4]: 'birth_date',
    df_isp.columns[5]: 'gender',
    df_isp.columns[6]: 'diagnosis',
}

# 左眼座標のマッピングを作成（X座標反転を考慮）
print("左眼座標のマッピングを作成中...")
left_eye_mapping = {}
for i, left_set in enumerate(left_eye_sets):
    if left_set['x_idx'] < len(df_isp.columns) and len(df_isp) > 0:
        # 左眼の座標を取得（最初の行で確認）
        x_val = df_isp.iloc[0, left_set['x_idx']]
        y_val = df_isp.iloc[0, left_set['y_idx']]
        
        # X座標を反転
        x_flipped = -x_val
        
        # 対応する右眼フォーマットのpoint番号を探す
        for j in range(28):
            if coord_isp_28.iloc[j]['x'] == x_flipped and coord_isp_28.iloc[j]['y'] == y_val:
                left_eye_mapping[i] = j  # 左眼のインデックスi → 右眼フォーマットのインデックスj
                break

print(f"左眼マッピング完了: {len(left_eye_mapping)}点")

# 全データを処理（両眼データを右眼・左眼に分けて整理）
print("全データを処理中...")
all_data = []

for idx in range(len(df_isp)):
    if idx % 1000 == 0:
        print(f"  {idx}/{len(df_isp)}行処理済み")
    
    row = df_isp.iloc[idx]
    
    # 基本情報を取得
    base_info = {new_name: row[old_name] for old_name, new_name in basic_cols.items()}
    
    # 右眼データ（最初の28点）
    right_data = extract_eye_data(row, right_eye_sets)
    row_data_right = base_info.copy()
    row_data_right['eye'] = 'OD'
    for i, val in enumerate(right_data):
        row_data_right[f'point{i+1}'] = val
    all_data.append(row_data_right)
    
    # 左眼データ（次の28点）
    left_data = extract_eye_data(row, left_eye_sets)
    row_data_left = base_info.copy()
    row_data_left['eye'] = 'OS'
    
    # 左眼データを右眼座標系に合わせて再配置
    reordered_left_data = [np.nan] * 28
    for left_idx, right_idx in left_eye_mapping.items():
        if left_idx < len(left_data):
            reordered_left_data[right_idx] = left_data[left_idx]
    
    for i, val in enumerate(reordered_left_data):
        row_data_left[f'point{i+1}'] = val
    all_data.append(row_data_left)

# DataFrameに変換
cleaned_df = pd.DataFrame(all_data)
print(f"\n処理完了！")
print(f"出力データ数: {len(cleaned_df)}")
print(f"右眼データ数: {len(cleaned_df[cleaned_df['eye'] == 'OD'])}")
print(f"左眼データ数: {len(cleaned_df[cleaned_df['eye'] == 'OS'])}")

左眼座標のマッピングを作成中...
左眼マッピング完了: 28点
全データを処理中...
  0/18379行処理済み
  1000/18379行処理済み
  2000/18379行処理済み
  3000/18379行処理済み
  4000/18379行処理済み
  5000/18379行処理済み
  6000/18379行処理済み
  7000/18379行処理済み
  8000/18379行処理済み
  9000/18379行処理済み
  10000/18379行処理済み
  11000/18379行処理済み
  12000/18379行処理済み
  13000/18379行処理済み
  14000/18379行処理済み
  15000/18379行処理済み
  16000/18379行処理済み
  17000/18379行処理済み
  18000/18379行処理済み

処理完了！
出力データ数: 36758
右眼データ数: 18379
左眼データ数: 18379


## 4. データ品質の確認と保存

In [23]:
# データ品質を確認
print("=== データ品質確認 ===")

# 各測定点の統計
measurement_cols = [f'point{i+1}' for i in range(28)]
for eye in ['OD', 'OS']:
    eye_data = cleaned_df[cleaned_df['eye'] == eye]
    if len(eye_data) > 0:
        print(f"\n{eye}（{'右眼' if eye == 'OD' else '左眼'}）:")
        all_vals = eye_data[measurement_cols].values.flatten()
        valid_vals = all_vals[~np.isnan(all_vals)]
        if len(valid_vals) > 0:
            print(f"  総測定値: {len(valid_vals)}")
            print(f"  見えた(1): {np.sum(valid_vals == 1)} ({np.sum(valid_vals == 1)/len(valid_vals)*100:.1f}%)")
            print(f"  見えない(0): {np.sum(valid_vals == 0)} ({np.sum(valid_vals == 0)/len(valid_vals)*100:.1f}%)")

# サンプルデータ表示
print("\n=== サンプルデータ ===")
display_cols = ['daily_id', 'date', 'eye'] + [f'point{i}' for i in range(1, 11)]
print(cleaned_df[display_cols].head(6))

=== データ品質確認 ===

OD（右眼）:
  総測定値: 514612
  見えた(1): 507329 (98.6%)
  見えない(0): 7283 (1.4%)

OS（左眼）:
  総測定値: 514612
  見えた(1): 507663 (98.6%)
  見えない(0): 6949 (1.4%)

=== サンプルデータ ===
   daily_id       date eye  point1  point2  point3  point4  point5  point6  \
0        49  2024/9/17  OD       1       1       1       1       1       1   
1        49  2024/9/17  OS       1       1       1       1       1       1   
2         4  2024/5/23  OD       1       1       1       1       1       1   
3         4  2024/5/23  OS       1       1       1       1       1       1   
4        59  2024/9/19  OD       1       1       1       1       1       1   
5        59  2024/9/19  OS       1       1       1       1       1       1   

   point7  point8  point9  point10  
0       1       1       1        1  
1       1       1       1        1  
2       1       1       1        1  
3       1       1       1        1  
4       1       1       1        1  
5       1       1       1        1  


In [24]:
# データを保存
import os

output_dir = 'outputs/hitachi_isp'
os.makedirs(output_dir, exist_ok=True)

# メインデータを保存
output_file = os.path.join(output_dir, 'isp_binocular_data.csv')
cleaned_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"データを保存: {output_file}")

# 座標参照を保存
coord_file = os.path.join(output_dir, 'coord_isp_28.csv')
coord_isp_28 = coord_isp.iloc[:28]
coord_isp_28.to_csv(coord_file)
print(f"座標データを保存: {coord_file}")

# レポート作成
report = f"""
# 日立ISPデータ処理レポート（両眼データのみ）
生成日: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## データ概要
- 入力行数: {len(df_isp)} (両眼データのみ)
- 出力行数: {len(cleaned_df)}
- 測定点数: 28（ISP標準配置）
- データ型: Bool (TRUE/FALSE → 1/0)

## 処理内容
- 検査眼が「両眼」のデータのみを抽出
- 各行から右眼28点、左眼28点を分離
- 右眼: 列53から28測定点（各5列）
- 左眼: 右眼の後の28測定点（各5列）
- TRUE → 1（見えた）
- FALSE → 0（見えない）
- 両眼とも同じpoint1-28形式で保存

## 注意事項
- coord_isp.csvの最初の28点がISP測定点
- 左眼データは右眼座標系に合わせて保存
"""

report_file = os.path.join(output_dir, 'processing_report_binocular.txt')
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(report)

print(f"\nレポートを保存: {report_file}")
print("\n処理完了！")

データを保存: outputs/hitachi_isp/isp_binocular_data.csv
座標データを保存: outputs/hitachi_isp/coord_isp_28.csv

レポートを保存: outputs/hitachi_isp/processing_report_binocular.txt

処理完了！
